In [ ]:
%%javascript
document.title='UC1 app'

In [ ]:
import asyncio
from io import BytesIO
from IPython.display import display
import json
from time import sleep

import ipywidgets as ipw
import matplotlib.pyplot as plt
import pandas as pd

from marketplace.app import get_app
from marketplace.client import MarketPlaceClient

In [ ]:
access_token = "MP_ACCESS_TOKEN"
# access_token = os.environ.get("MP_ACCESS_TOKEN")
mp_client = MarketPlaceClient(
    marketplace_host_url="https://materials-marketplace.eu/",
    access_token=access_token,
)

In [ ]:
simpartix = get_app(client=mp_client, app_id="7ba1257a-0d38-4479-9606-124d01ddae35")
micress = get_app(client=mp_client, app_id="4a74a6b1-7a0f-46af-8f76-c7d87683d79a")

In [ ]:
class Labeled(ipw.HBox):
    def __init__(self, widget, description, unit=""):
        layout = ipw.Layout(width="212px", flex_flow="column", align_items="flex-end")
        super(Labeled, self).__init__(
            [
                ipw.HBox(children=[ipw.Label(description)], layout=layout),
                widget,
                ipw.Label(unit),
            ]
        )

In [ ]:
class Step(ipw.VBox):
    def __init__(self, simulation, children, **kwargs):
        self.simulation = simulation
        self.transformation_id = ""

        self.style = {"red": "#bc1320", "green": "#27a22d", "blue": "#1e4f7e"}
        self.state = ipw.Label()
        self.start = ipw.Button(
            description="Start", icon="play", button_style="success"
        )
        self.start.style.button_color = self.style["green"]
        self.stop = ipw.Button(
            description="Stop", icon="stop", button_style="warning", disabled=True
        )
        self.stop.style.button_color = self.style["red"]
        self.output = ipw.Output()

        super().__init__(children=[*children, ipw.HBox([self.start, self.stop]),
            self.state, self.output], **kwargs)


    def _refresh(self, _=None):
        if self.transformation_id:
            self.state.value = self.simulation.get_transformation_state(
                self.transformation_id
            ).value

    def _poll_refresh(self, timeout=300):
        async def poll():
            
            with self.output:
                print("test1")
            self._refresh()
            while self.state.value == "RUNNING":
                await self._update_ui()
                sleep(3)
                self._refresh()
            await self._update_ui()

        asyncio.ensure_future(asyncio.wait_for(poll(), timeout))

    async def _update_ui(self):
        if self.state.value == "RUNNING":
            self.start.disabled = True
            self.stop.disabled = False
        else:
            self.start.disabled = False
            self.stop.disabled = True

        await self._update_output()

    def _stop(self, _=None):
        self.simulation.stop_transformation(self.transformation_id)
    

In [ ]:
class SimPartixStep(Step):
    def __init__(self, *args, **kwargs):
        self.laser_power_selector = ipw.FloatSlider(
            min=10, max=300, step=10, value=200,
            layout=ipw.Layout(min_width="200px"))
        self.laser_speed_selector = ipw.FloatSlider(
            min=0, max=4, step=0.1, value=3,
            layout=ipw.Layout(min_width="200px"))
        self.phi = ipw.FloatSlider(
            min=0, max=1, step=0.1, value=0.7,
            layout=ipw.Layout(min_width="200px"))
        self.powder_layer_height = ipw.FloatSlider(
            min=3.00e-5, max=1.00e-4, step=1.00e-5, value=6.00e-5, readout_format='.2e',
            layout=ipw.Layout(min_width="200px"))
        self.sphere_diameter = ipw.FloatSlider(
            min=1.5e-5, max=3e-5, step=5e-6, value=3e-5, readout_format='.2e',
            layout=ipw.Layout(min_width="200px"))
        
        # self.mapping_button = ipw.Button(description="Semantic Mapping")
        
        
        super().__init__(simulation=simpartix,
            children=[
            ipw.Label('Configure the input parameters and click on "Submit" to start the simulation. '
                      '(Default values are shown)'),
            Labeled(self.laser_power_selector, description="Laser power:", unit="W"),
            Labeled(self.laser_speed_selector, description="Laser speed:", unit="m/s"),
            Labeled(self.phi, description="Filling fraction:", unit="-"),
            Labeled(self.powder_layer_height, description="Powder layer height:", unit="m"),
            Labeled(self.sphere_diameter, description="Sphere diameter:", unit="m"),
            # self.mapping_button,
        ],
            *args, **kwargs)
        
        self.start.on_click(self._start)
        self.stop.on_click(self._stop)
        # self.mapping_button.on_click(self._semantic_mapping)
        
    # def _semantic_mapping(self, _=None):
    #     with self.output:
    #         mapping = self.simulation.get_semantic_mapping("SimpartixOutput")
    #         display(ipw.Label("Semantic mapping:"))
    #         print(json.dumps(mapping, indent=4, sort_keys=True))
        
    def _start(self, _=None):
        if not self.transformation_id:
            simulation = self.simulation.new_transformation({'laserPower': self.laser_power_selector.value,
                                                      'laserSpeed': self.laser_speed_selector.value,
                                                      'phi': self.phi.value,
                                                      'powderLayerHeight': self.powder_layer_height.value,
                                                      'sphereDiameter': self.sphere_diameter.value})
            self.transformation_id = str(simulation.id)
        self.simulation.start_transformation(self.transformation_id)
        self._poll_refresh()
        

    async def _update_output(self):
        try:
            if self.state.value == "COMPLETED":
                transformation = simpartix.get_transformation(self.transformation_id)
                self.output.append_stdout(transformation.json())
        except Exception as error:
            with self.output:
                display(f"Failed to update output: {error}")


In [ ]:
from IPython.display import Image
class MicressStep(Step):
    def __init__(self, *args, **kwargs):
        self.simulation = None
        
        self.microstructure = [
            3, 3, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3,
            3, 3, 3, 3, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3,
            2, 2, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
            2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 2, 2, 3, 3, 1, 1, 2, 2,
            2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, 1,
            0, 0, 0, 0, 0, 0, 2, 2, 3, 3, 1, 1, 2, 2, 2, 2, 0, 0,
            0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2,
            2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
            1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
            2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2,
            3, 3, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 3, 3, 2, 2,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 3, 2, 2,
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 3, 3, 3,
            0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 3, 2, 2, 0, 0, 0, 0,
            0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3, 3, 3, 3, 0, 0, 0, 0
        ]
        self.microstructure_size_x = 18
        self.microstructure_size_z = 32

        self.initial_temperature = ipw.IntSlider(min=1, max=2000, value=1250)
        self.temperature_gradient = ipw.FloatSlider(
            min=-10, max=10, value=0.1, step=0.1
        )
        self.cooling_rate = ipw.FloatSlider(min=-100, max=100, value=-50, step=1)        

        children = [
            ipw.Label(
                "Use the following sliders to adjust the initial temperature, temperature gradient and cooling rate. Click the Start button to run the simulation.",
            ),
            Labeled(
                self.initial_temperature, description="Initial temperature:", unit="$K$"
            ),
            Labeled(
                self.temperature_gradient,
                description="Temperature gradient:",
                unit="$K\over \mu m$",
            ),
            Labeled(self.cooling_rate, description="Cooling rate:", unit="$K\over s$"),
        ]
        super().__init__(simulation=micress, children=children, *args, **kwargs)

        self.start.on_click(self._start)


    def _start(self, _):
        params = {
            "parameters": {
                "microstructure": self.microstructure,
                "microstructure_size_x": self.microstructure_size_x,
                "microstructure_size_z": self.microstructure_size_z,
                "initial_temperature": self.initial_temperature.value,
                "temperature_gradient": self.temperature_gradient.value,
                "cooling_rate": self.cooling_rate.value,
            },
            "state": "RUNNING",
        }
        self.transformation_id = micress.new_transformation(params).id
        self._poll_refresh()

    def _output(self, text, clear=True):
        with self.output:
            self.output.clear_output(wait=True)
            display(text)

    def _plot(self, parameters, clear=True):
        timesteps = parameters["timesteps"]
        df = pd.DataFrame(timesteps)
        self.output.clear_output(wait=True)
        df = pd.DataFrame(timesteps)
        ax = df.plot(
            x="temperature",
            y=["liquid_fraction", "solid_fraction"],
            label=["Liquid", "Solid"],
            xlabel="Temperature $[K]$",
            ylabel="Fraction",
            title="Phase Fractions",
            figsize=(9, 6),
            style=["o--", "o-"],
            color=[self.style["red"], self.style["green"]],
        )
        ax.invert_xaxis()
        plt.savefig('plot.png')
        self.output.append_display_data(Image(filename="plot.png"))

    def _refresh_file(self, timeout=300):
        asyncio.ensure_future(self._fetch_output())

    async def _update_output(self):
        try:
            transformation = micress.get_transformation(self.transformation_id)
            if (
                "timesteps" in transformation.parameters
                and transformation.parameters["timesteps"]
            ):
                self._plot(transformation.parameters)
            else:
                with self.output:
                    display(transformation)
        except Exception as error:
            self._output(f"Failed to update output: {error}")


In [ ]:
simpartix_step = SimPartixStep()
micress_step = MicressStep()

accordion = ipw.Accordion([simpartix_step, micress_step])

accordion.set_title(0, "SimPARTIX®")
accordion.set_title(1, "MICRESS®")

app = ipw.VBox(
    [
        ipw.HTML(
            '<img src="logo.png" alt="UC1 logo" class="center" style="width: 300px; padding-bottom: 20px;"/>'
        ),
        ipw.HTML(
            "<p>Use Case 1 (UC1) app simulating the solidification process during selective laser melting (SLM).</p>"
        ),
        accordion,
    ]
)


display(app)